# Beer and USA

## imports

In [6]:
from src.data.save_tar_gz import tar_gz_to_csv
from src.data.load_data import load_data_from_csv
from src.utils.data_utils import *
from src.data.additional_data import *
import warnings
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import src.data.wrangling as wrangling

/Users/petertaylor/Documents/Info/master/Ada/ada-2024-project-fandemondada/src/data/additional_data.py:36: DeprecationWarning: invalid escape sequence '\('
  .apply(lambda x : x.str.split(' \(r').str[0])


## Loading the data into dict. of dataframes

The first option loads the data from the compressed .tar.gz files, the second option loads directly from extracted csv files.

In [ ]:
# put the tar.tz files in the data/ folder
# warnings.filterwarnings("ignore")
# data = tar_gz_to_csv(load_path= 'data/', save_path='data/raw/', load_text = False)
# warnings.filterwarnings("default")

##use this line if you have all the data extracted to csv files under data/raw in subfolders BeerAdvocate, RateBeer, and MatchedBeerData
##it loads the data into 3 dictionnaries containing the dataframes
data_ba, data_rb, data_matched = load_data_from_csv('data/raw/')


In [ ]:
print(data_ba.keys())
print(data_rb.keys())
print(data_matched.keys())

#remove textual review columns if needed
remove_txt_columns(data_ba, data_rb, data_matched)

# Cleaning and wrangling the data

For each dataset, we will clean the data by removing duplicates, missing values, and irrelevant columns.
To remove missing values we'll use the following strategy:
We'll extract a new column containing a boolean value indicating if the row has missing values. And replace the missing values by 0. This can be useful in a logistic regression if having a missing value at field x actually yields some information.

In [ ]:
data_path = "data/raw"
Ratebeer_path = lambda path: f"{path}/Ratebeer"
BeerAdvocate_path = lambda path: f"{path}/BeerAdvocate"
Matched_path = lambda path: f"{path}/MatchedBeerData"
clean_path = "data/clean"

### Cleaning BeerAdvocate data

We'll start by cleaning the BeerAdvocate data. We'll clean the users, ratings, beers, and breweries data.
For users, we'll only focus on american users. 
We'll filter the users by their location and keep only the ones that have a location in the USA. Same with ratings we'll remove those not made by american users. We also remove the text reviews as we won't use them. Finally the missing data are handled as explained above.

In [ ]:
wrangling.clean_beer_advocate(
    raw_data_path=BeerAdvocate_path(data_path),
    clean_data_path=BeerAdvocate_path(clean_path)
)

### Cleaning RateBeer data

The cleaning process is exactly the same as for BeerAdvocate

In [ ]:
wrangling.clean_ratebeer(
    raw_data_path=Ratebeer_path(data_path),
    clean_data_path=Ratebeer_path(clean_path)
)

### Cleaning Matched data

Here the cleaning is a bit different as we have to match the data from BeerAdvocate and RateBeer.
As the columns have multiple types (the first row contains the name of the column in the original dataset), we have to clean them first. We'll delete the first row and append it to the column names so that they are more readable (for example ra_1 becomes ra_text). We'll then convert the columns to the correct type and remove the rows with missing values. The rest of the cleaning process is the same as for the other datasets.

In [ ]:
wrangling.clean_matched_data(
    raw_data_path=Matched_path(data_path),
    clean_data_path=Matched_path(clean_path)
)

## Analyse distribution of US users per state

In [ ]:
users_matched = data_matched['matched_beer_data_users.csv']
users_ba = data_ba['BeerAdvocate_users.csv']
users_rb = data_rb['RateBeer_users.csv']
users_ba = users_ba.dropna(subset=['location'])
users_rb = users_rb.dropna(subset=['location'])

In [ ]:
users_matched_usa = users_matched[users_matched['ba.1'].str.startswith('United States')]
users_ba_usa = users_ba[users_ba['location'].str.startswith('United States')]
users_rb_usa = users_rb[users_rb['location'].str.startswith('United States')]


print(f"Number of matched users in USA: {users_matched_usa.shape[0]}")
print(f"Number of BeerAdvocate users in USA: {users_ba_usa.shape[0]}")
print(f"Number of RateBeer users in USA: {users_rb_usa.shape[0]}")

In [ ]:
states_usa_ba = users_ba_usa['location'].apply(lambda x: x.split(', ')[1])
states_usa_rb = users_rb_usa['location'].apply(lambda x: x.split(', ')[1])

# print(f"USA states represented in BeerAdvocate: {len(states_usa_ba.unique())}")
# print(f"USA states represented in RateBeer: {len(states_usa_rb.unique())}")

# print(f"Number of users per state in BeerAdvocate: {states_usa_ba.value_counts()}")
# print(f"Number of users per state in RateBeer: {states_usa_rb.value_counts()}")

users_per_state_ba = states_usa_ba.value_counts().to_dict()
users_per_state_rb = states_usa_rb.value_counts().to_dict()
users_per_state_barb = {k: (users_per_state_ba[k] + users_per_state_rb[k]) for k in users_per_state_ba.keys()}

states = users_per_state_barb.keys()
users = users_per_state_barb.values()

fig, ax = plt.subplots(figsize=(15, 8))
ax.bar(states, users)
ax.set_ylabel('Number of users')
ax.set_xlabel('States')
plt.xticks(rotation=90)
plt.show()



## Analyse distribution of US Breweries per state

In [ ]:
breweries_matched = data_matched['matched_beer_data_breweries.csv']
breweries_ba = data_ba['BeerAdvocate_breweries.csv']
breweries_rb = data_rb['RateBeer_breweries.csv']
breweries_ba = breweries_ba.dropna(subset=['location'])
breweries_rb = breweries_rb.dropna(subset=['location'])

In [ ]:
breweries_matched_usa = breweries_matched[breweries_matched['ba.1'].str.startswith('United States')]
breweries_ba_usa = breweries_ba[breweries_ba['location'].str.startswith('United States')]
breweries_rb_usa = breweries_rb[breweries_rb['location'].str.startswith('United States')]


print(f"Number of matched breweries in USA: {breweries_matched_usa.shape[0]}")
print(f"Number of BeerAdvocate breweries in USA: {breweries_ba_usa.shape[0]}")
print(f"Number of RateBeer breweries in USA: {breweries_rb_usa.shape[0]}")

In [ ]:
breweries_name_split_ba = breweries_ba_usa['location'].apply(lambda x: len(x.split(',')))
breweries_filtered_ba = breweries_ba_usa[breweries_name_split_ba == 2]
breweries_filtered_ba = breweries_filtered_ba[breweries_filtered_ba['location'] != 'United States, United States']
breweries_per_states_ba = breweries_filtered_ba['location'].apply(lambda x: x.split(',')[1])

breweries_name_split_rb = breweries_rb_usa['location'].apply(lambda x: len(x.split(',')))
breweries_filtered_rb = breweries_rb_usa[breweries_name_split_rb == 2]
breweries_filtered_rb['location'] = breweries_filtered_rb['location'].mask(breweries_filtered_rb['location'] == 'United States, Washington DC', 'United States, California')
breweries_per_states_rb = breweries_filtered_rb['location'].apply(lambda x: x.split(',')[1])

# print(f"USA states with breweries represented in BeerAdvocate: {len(breweries_per_states_ba.unique())}")
# print(f"USA states with breweries represented in RateBeer: {len(breweries_per_states_rb.unique())}")

# print(f"Number of breweries per state in BeerAdvocate: {breweries_per_states_ba.value_counts()}")
# print(f"Number of breweries per state in RateBeer: {breweries_per_states_rb.value_counts()}")

breweries_per_state_ba = breweries_per_states_ba.value_counts().to_dict()
breweries_per_state_rb = breweries_per_states_rb.value_counts().to_dict()

#delete the space at the start of each key
breweries_per_state_ba = {k[1:]: v for k, v in breweries_per_state_ba.items()}
breweries_per_state_rb = {k[1:]: v for k, v in breweries_per_state_rb.items()}

breweries_per_state_barb = {k: (breweries_per_state_ba[k] + breweries_per_state_rb[k]) for k in breweries_per_state_ba.keys()}

states = breweries_per_state_barb.keys()
users = breweries_per_state_barb.values()

fig, ax = plt.subplots(figsize=(15, 8))
ax.bar(states, users)
ax.set_ylabel('Number of users')
ax.set_xlabel('States')
plt.xticks(rotation=90)
plt.show()



## Analyse ratio of users per brewery in each state

In [ ]:
#Amount of users per brewery in each state
users_per_brewery_barb = {k: (users_per_state_barb[k] / breweries_per_state_barb[k]) for k in users_per_state_ba.keys()}
print(users_per_brewery_barb)

#sort so easier to view plot
sorted_users_per_brewery_ba = dict(sorted(users_per_brewery_barb.items(), key=lambda item: item[1]))

states = sorted_users_per_brewery_ba.keys()
users_per_brewery = sorted_users_per_brewery_ba.values()

fig, ax = plt.subplots(figsize=(15, 8))
ax.bar(states, users_per_brewery)
ax.set_ylabel('Number of users per brewery')
ax.set_xlabel('States')
plt.xticks(rotation=90)
plt.show()

### Data from Census

In [ ]:
census_path = 'data/raw/Census/'
file_name_2020 = 'DECENNIALCD1182020.H2_rural_urban_US.csv'
file_name_2010 = 'DECENNIALCD1162010.H2_rural_urban_US.csv'

urban_2020_df = load_urban_frac_df(census_path, file_name_2020).reset_index()
urban_2010_df = load_urban_frac_df(census_path, file_name_2010).reset_index()

print(urban_2020_df.head())
print(urban_2010_df.head())

## Number of users per habitant in each state
This allows us to see if we have enough users per state, compared to its population.

In [ ]:
if all(col in urban_2010_df.columns for col in ['urban_pop', 'rural_pop', 'urban_frac']):
    urban_2010_df = urban_2010_df.drop(columns=['urban_pop', 'rural_pop', 'urban_frac'])
urban_2010_dict = urban_2010_df.set_index('state').to_dict()['total_pop']

# delete keys states that are not in the 50 official
urban_2010_dict = {k: v for k, v in urban_2010_dict.items() if k in users_per_state_barb.keys()}

# number of habitants per user in each state
users_per_habitant = {k: (urban_2010_dict[k] / users_per_state_barb[k]) for k in users_per_state_barb.keys()}
print(users_per_habitant)

states = users_per_habitant.keys()
users_per_habitant = users_per_habitant.values()

fig, ax = plt.subplots(figsize=(15, 8))
ax.bar(states, users_per_habitant)
ax.set_ylabel('Number of users per habitant')
ax.set_xlabel('States')
plt.xticks(rotation=90)
plt.show()

## Alcohol consumption per capita per state

In [ ]:
icpsr_path = 'data/raw/OPENICPSR/OPENICPSR_apparent_per_capita_alcohol_consumption.csv'
data_icpsr = load_icpsr(icpsr_path) #load + format data
print(data_icpsr.keys())
data_icpsr['2002'].head(5)

## age distribution per state

In [ ]:
age_path = 'data/raw/GeneralPopulationAge/DECENNIALDP2020.DP1-2024-11-14T134434.csv'
age_total, age_male, age_female = load_age_data(age_path, load_gender = True)
age_total.head(5)